In [1]:
pip install -U pygame --user

Requirement already up-to-date: pygame in /Users/rishichalla/.local/lib/python3.8/site-packages (2.0.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pygame, sys
from pygame.locals import *
import io
from urllib.request import urlopen
from random import randint

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
#conda install -c cogsci pygame

In [4]:
pygame.init()

(7, 0)

In [5]:
screen = pygame.display.set_mode((800, 600))

In [6]:
running = True

In [7]:
#Sprites
icon = pygame.image.load('./duck.png')

In [8]:
#Backgrounds
bg = pygame.image.load("./qwackBg.jpg")
level0 = pygame.image.load("./level0.jpg")

In [9]:
#Global Variables
pygame.display.set_caption("Qwack")
meme = pygame.image.load('./qwackMeme.jpg')
pygame.display.set_icon(meme)

fontTitle = pygame.font.SysFont("Comic", 125)
fontSubHeader = pygame.font.SysFont(None, 50)
text = pygame.font.SysFont(None, 25)

mainClock = pygame.time.Clock()

click = False
themeColor = (255,255,0)


In [10]:
def draw_text(text, font, color, surface, x, y):
    textObj = font.render(text,1,color)
    textRect = textObj.get_rect()
    textRect.topleft = (x,y)
    surface.blit(textObj, textRect)
    return textRect

In [11]:
#icon
global iconX, iconY, velocity
iconX = 370
iconY = 480
velocity = [randint(3,4),randint(-4,4)]

In [12]:
def Icon(sprite, x,y):
    screen.blit(sprite, (x, y))

In [13]:
def main_menu(iconX, iconY):
    while True:
        
        # Setup
        screen.blit(bg, (0,0))
        mx, my = pygame.mouse.get_pos()
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        Icon(icon, iconX, iconY)
        
        # Buttons
        button_1 = pygame.Rect(65, 140, 230, 45)
        button_2 = pygame.Rect(65, 215, 230, 45)
        button_3 = pygame.Rect(65, 290, 230, 45)
        
        if button_1.collidepoint((mx, my)):
            if click:
                level_select(iconX, iconY)
        if button_2.collidepoint((mx, my)):
            if click:
                Controls(iconX, iconY)
        if button_3.collidepoint((mx, my)):
            if click:
                Credits(iconX, iconY)
                
        pygame.draw.rect(screen, themeColor, button_1)
        pygame.draw.rect(screen, themeColor, button_2)
        pygame.draw.rect(screen, themeColor, button_3)
        
        # Text
        draw_text("Qwack", fontTitle, themeColor, screen, 260, 15)
        draw_text("Start Game", fontSubHeader, (0, 0, 0), screen, 75, 150)
        draw_text("Controls", fontSubHeader, (0, 0, 0), screen, 75, 225)
        draw_text("Credits", fontSubHeader, (0, 0, 0), screen, 75, 300)
        
        # Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    pygame.quit()
                    sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
                    # print(mx, my)
        pygame.display.update()
        mainClock.tick(60) 

In [14]:
def level_select(iconX, iconY):
    running = True
    while running:
        
        #Initialize
        screen.fill((0,0,0))
        screen.blit(bg, (0,0))
        mx, my = pygame.mouse.get_pos()
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        Icon(icon, iconX, iconY)
        
        draw_text("Qwack", fontTitle, themeColor, screen, 262, 15)
        draw_text("Level Select", fontSubHeader, (255, 255, 255), screen, 307, 110)
        draw_text("___________", fontSubHeader, (255, 255, 255), screen, 307.25, 112)
        draw_text("Press Esc To Return", text, themeColor, screen, 10, 15)
        buttonLevel_0 = draw_text("Level 0: Tutorial", text, (255, 255, 255), screen, 75, 175)
        buttonLevel_1 = draw_text("Level 1: Entanglement", text, (255, 255, 255), screen, 75, 225)
        buttonLevel_2 = draw_text("Level 2:", text, (255, 255, 255), screen, 75, 275)
        buttonLevel_3 = draw_text("Level 3:", text, (255, 255, 255), screen, 75, 325)
        buttonLevel_4 = draw_text("Level 4:", text, (255, 255, 255), screen, 75, 375)
        buttonLevel_5 = draw_text("Level 5:", text, (255, 255, 255), screen, 75, 425)
        buttonLevel_6 = draw_text("Level 6:", text, (255, 255, 255), screen, 75, 475)
        
        #player
        playerX = 370
        playerY = 490
        xChange = 0
        yChange = 0
        
        
            
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
                    
        if buttonLevel_0.collidepoint((mx, my)):
            if click:
                level_0(playerX, playerY, xChange, yChange)
        if buttonLevel_1.collidepoint((mx, my)):
            if click:
                pass
        if buttonLevel_2.collidepoint((mx, my)):
            if click:
                pass
        pygame.display.update()
        mainClock.tick(60)

In [15]:
def Controls(iconX, iconY):
    running = True
    while running:
        screen.fill((0,0,0))
        screen.blit(bg, (0,0))
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        Icon(icon, iconX, iconY)
        
        draw_text("Qwack", fontTitle, themeColor, screen, 262, 15)
        draw_text("Press Esc To Return", text, themeColor, screen, 10, 15)
        draw_text("Controls", fontSubHeader, (255, 255, 255), screen, 307, 110)
        draw_text("________", fontSubHeader, (255, 255, 255), screen, 307.25, 112)
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                    
        pygame.display.update()
        mainClock.tick(60) 

In [16]:
def Credits(iconX, iconY):
    running = True
    while running:
        screen.fill((0,0,0))
        screen.blit(bg, (0,0))
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        Icon(icon, iconX, iconY)
        
        draw_text("Qwack", fontTitle, themeColor, screen, 262, 15)
        draw_text("Press Esc To Return", text, themeColor, screen, 10, 15)
        draw_text("About Us", fontSubHeader, (255, 255, 255), screen, 307, 110)
        draw_text("________", fontSubHeader, (255, 255, 255), screen, 307.25, 112)
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                    
        pygame.display.update()
        mainClock.tick(60) 

In [17]:
def level_0(x, y, xChange, yChange):
    running = True
    jump = False
    i = 0
    while running:
        # Setup
        screen.fill((0,0,0))
        screen.blit(level0, (0,0))
        mx, my = pygame.mouse.get_pos()
        instructions = ["Hello! Welcome to Qwack!", "This is a game about exploring and learning Quantum Computing Concepts",
                       "Meet Dr. Qwack! He is stuck in a quantum realm and needs your help to escape!",
                       "Use the LEFT ARROW and RIGHT ARROW keys to move",
                       "Use SPACE to Jump",
                       "If at any point you want to abandon Dr. Qwack, press ESC"]
        #Instructions
        if i == 0:
            draw_text("Use the UP Arrow to continue reading...", fontSubHeader, (255, 255, 255), screen, 120, 200)
        draw_text(instructions[i], text, (255, 255, 255), screen, 120, 400)
    
      
        #Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                # movement
                if event.key == pygame.K_LEFT:
                    xChange = -5
                if event.key == pygame.K_RIGHT:
                    xChange = 5
                if event.key == pygame.K_SPACE and jump is False:
                    jump = True
                    yChange = 15
                if event.key == pygame.K_UP:
                    i += 1
                if event.key == pygame.K_DOWN:
                    i -= 1
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    xChange = 0
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
                    # print(mx, my)
                    
        # Set movement Values
        x += xChange
        if x <= 0:
            x = 0
        elif x >= 780:
            x = 780
            
        if jump:
            y -= yChange
            yChange -= 1
            if yChange < -15:
                jump = False
                yChange = 0
        
        #Display
        Icon(icon, x, y)
        pygame.display.update()
        mainClock.tick(60) 

In [18]:
# Game Loop
main_menu(iconX, iconY)


IndexError: list index out of range

In [ ]:
''' Formatting changes to make:

    Menu
    -Start
        -Level Select
        -Level 0 (story intro, tutorial)
    -Controls
    -Credits
    
    add note about "Hit Esc to go back"
    
    Color scheme??
'''